# 2. ‡∏Å‡∏≤‡∏£‡∏™‡∏£‡πâ‡∏≤‡∏á Agent (Agent Creation)
## ‡∏Ç‡∏±‡πâ‡∏ô‡∏ï‡∏≠‡∏ô‡∏Å‡∏≤‡∏£‡∏™‡∏£‡πâ‡∏≤‡∏á RL Agent ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö Crypto Trading

### ‡πÄ‡∏õ‡πâ‡∏≤‡∏´‡∏°‡∏≤‡∏¢:
- ‡πÇ‡∏´‡∏•‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏µ‡πà‡∏õ‡∏£‡∏∞‡∏°‡∏ß‡∏•‡∏ú‡∏•‡πÅ‡∏•‡πâ‡∏ß
- ‡∏™‡∏£‡πâ‡∏≤‡∏á Trading Environment
- ‡∏Å‡∏≥‡∏´‡∏ô‡∏î Agent Architecture
- ‡∏ï‡∏±‡πâ‡∏á‡∏Ñ‡πà‡∏≤ Hyperparameters
- ‡πÄ‡∏ï‡∏£‡∏µ‡∏¢‡∏° Agent ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö‡∏Å‡∏≤‡∏£‡πÄ‡∏ó‡∏£‡∏ô

## Cell 1: Import Libraries ‡πÅ‡∏•‡∏∞ Setup

In [4]:
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from datetime import datetime
import ta  # Technical Analysis library
from sklearn.preprocessing import MinMaxScaler

# Import config
from config import *

# Setup directories
RAW_DIR = "raw_data"
PROCESSED_DIR = "processed_data"

if not os.path.exists(PROCESSED_DIR):
    os.makedirs(PROCESSED_DIR)

print("üìÅ Setup directories completed")
print(f"üìä Starting Data Preprocessing Process")

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

üìÅ Setup directories completed
üìä Starting Data Preprocessing Process


## Cell 2: ‡πÇ‡∏´‡∏•‡∏î‡πÅ‡∏•‡∏∞‡∏ó‡∏≥‡∏Ñ‡∏ß‡∏≤‡∏°‡∏™‡∏∞‡∏≠‡∏≤‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•

In [5]:
def load_raw_data():
    """
    ‡πÇ‡∏´‡∏•‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏î‡∏¥‡∏ö‡∏à‡∏≤‡∏Å‡πÑ‡∏ü‡∏•‡πå CSV
    """
    print("üìÇ Loading raw data...")
    
    # ‡∏´‡∏≤‡πÑ‡∏ü‡∏•‡πå‡∏•‡πà‡∏≤‡∏™‡∏∏‡∏î‡πÉ‡∏ô raw_data
    csv_files = [f for f in os.listdir(RAW_DIR) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in raw_data directory")
    
    latest_file = max(csv_files, key=lambda x: os.path.getctime(os.path.join(RAW_DIR, x)))
    file_path = os.path.join(RAW_DIR, latest_file)
    
    # ‡πÇ‡∏´‡∏•‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    print(f"‚úÖ Loaded {len(df)} rows from {latest_file}")
    print(f"üìÖ Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    print(f"üìä Symbols: {df['tic'].unique()}")
    
    return df

def clean_data(df):
    """
    ‡∏ó‡∏≥‡∏Ñ‡∏ß‡∏≤‡∏°‡∏™‡∏∞‡∏≠‡∏≤‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
    """
    print("üßπ Cleaning data...")
    
    # ‡∏•‡∏ö‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ã‡πâ‡∏≥
    initial_len = len(df)
    df = df.drop_duplicates()
    print(f"  Removed {initial_len - len(df)} duplicate rows")
    
    # ‡πÄ‡∏£‡∏µ‡∏¢‡∏á‡∏•‡∏≥‡∏î‡∏±‡∏ö‡∏ï‡∏≤‡∏°‡πÄ‡∏ß‡∏•‡∏≤
    df = df.sort_values(['timestamp', 'tic'])
    df = df.reset_index(drop=True)
    
    # ‡∏ï‡∏£‡∏ß‡∏à‡∏™‡∏≠‡∏ö missing values
    missing_values = df.isnull().sum()
    if missing_values.any():
        print("\n‚ö†Ô∏è Found missing values:")
        print(missing_values[missing_values > 0])
        
        # ‡πÅ‡∏Å‡πâ‡πÑ‡∏Ç missing values
        for col in df.columns:
            if df[col].isnull().any():
                if col in ['open', 'high', 'low', 'close']:
                    # Forward fill ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö‡∏£‡∏≤‡∏Ñ‡∏≤
                    df[col] = df.groupby('tic')[col].fillna(method='ffill')
                elif col == 'volume':
                    # ‡πÅ‡∏ó‡∏ô‡∏ó‡∏µ‡πà‡∏î‡πâ‡∏ß‡∏¢ 0 ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö volume
                    df[col] = df[col].fillna(0)
    
    # ‡∏ï‡∏£‡∏ß‡∏à‡∏™‡∏≠‡∏ö‡∏Ñ‡πà‡∏≤‡∏ú‡∏¥‡∏î‡∏õ‡∏Å‡∏ï‡∏¥
    numeric_columns = ['open', 'high', 'low', 'close', 'volume']
    for col in numeric_columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        if len(outliers) > 0:
            print(f"\n‚ö†Ô∏è Found {len(outliers)} outliers in {col}")
            # ‡πÅ‡∏ó‡∏ô‡∏ó‡∏µ‡πà‡∏Ñ‡πà‡∏≤‡∏ú‡∏¥‡∏î‡∏õ‡∏Å‡∏ï‡∏¥‡∏î‡πâ‡∏ß‡∏¢‡∏Ñ‡πà‡∏≤‡πÉ‡∏Å‡∏•‡πâ‡πÄ‡∏Ñ‡∏µ‡∏¢‡∏á
            df.loc[df[col] < lower_bound, col] = lower_bound
            df.loc[df[col] > upper_bound, col] = upper_bound
    
    print("‚úÖ Data cleaning completed")
    return df

# ‡πÇ‡∏´‡∏•‡∏î‡πÅ‡∏•‡∏∞‡∏ó‡∏≥‡∏Ñ‡∏ß‡∏≤‡∏°‡∏™‡∏∞‡∏≠‡∏≤‡∏î‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
df = load_raw_data()
df = clean_data(df)

print(f"\nüìä Cleaned data summary:")
print(f"  Total rows: {len(df)}")
print(f"  Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"  Symbols: {df['tic'].unique()}")

üìÇ Loading raw data...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'raw_data'

## Cell 3: ‡∏Ñ‡∏≥‡∏ô‡∏ß‡∏ì Technical Indicators

In [ ]:
def calculate_technical_indicators(df):
    """
    ‡∏Ñ‡∏≥‡∏ô‡∏ß‡∏ì Technical Indicators ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö‡πÅ‡∏ï‡πà‡∏•‡∏∞ symbol
    """
    print("üìà Calculating technical indicators...")
    
    # ‡∏™‡∏£‡πâ‡∏≤‡∏á DataFrame ‡πÉ‡∏´‡∏°‡πà
    processed_df = pd.DataFrame()
    
    # ‡∏Ñ‡∏≥‡∏ô‡∏ß‡∏ì indicators ‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö‡πÅ‡∏ï‡πà‡∏•‡∏∞ symbol
    for symbol in df['tic'].unique():
        print(f"\nProcessing {symbol}...")
        symbol_data = df[df['tic'] == symbol].copy()
        
        # Trend Indicators
        symbol_data['sma_20'] = ta.trend.sma_indicator(symbol_data['close'], window=20)
        symbol_data['sma_50'] = ta.trend.sma_indicator(symbol_data['close'], window=50)
        symbol_data['ema_20'] = ta.trend.ema_indicator(symbol_data['close'], window=20)
        symbol_data['macd'] = ta.trend.macd_diff(symbol_data['close'])
        
        # Momentum Indicators
        symbol_data['rsi'] = ta.momentum.rsi(symbol_data['close'], window=14)
        symbol_data['stoch'] = ta.momentum.stoch(symbol_data['high'], symbol_data['low'], symbol_data['close'])
        symbol_data['williams_r'] = ta.momentum.williams_r(symbol_data['high'], symbol_data['low'], symbol_data['close'])
        
        # Volatility Indicators
        symbol_data['bb_high'] = ta.volatility.bollinger_hband(symbol_data['close'])
        symbol_data['bb_low'] = ta.volatility.bollinger_lband(symbol_data['close'])
        symbol_data['atr'] = ta.volatility.average_true_range(symbol_data['high'], symbol_data['low'], symbol_data['close'])
        
        # Volume Indicators
        symbol_data['obv'] = ta.volume.on_balance_volume(symbol_data['close'], symbol_data['volume'])
        symbol_data['vwap'] = ta.volume.volume_weighted_average_price(symbol_data['high'], symbol_data['low'], symbol_data['close'], symbol_data['volume'])
        
        # ‡πÄ‡∏û‡∏¥‡πà‡∏°‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏µ‡πà‡∏Ñ‡∏≥‡∏ô‡∏ß‡∏ì‡πÅ‡∏•‡πâ‡∏ß
        processed_df = pd.concat([processed_df, symbol_data])
        
        print(f"‚úÖ Added {len(symbol_data)} rows with indicators")
    
    # ‡πÄ‡∏£‡∏µ‡∏¢‡∏á‡∏•‡∏≥‡∏î‡∏±‡∏ö‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
    processed_df = processed_df.sort_values(['timestamp', 'tic']).reset_index(drop=True)
    
    print("\n‚úÖ Technical indicators calculation completed")
    print(f"üìä Total indicators: {len(processed_df.columns) - 6}")  # ‡∏•‡∏ö 6 ‡∏Ñ‡∏≠‡∏•‡∏±‡∏°‡∏ô‡πå‡∏û‡∏∑‡πâ‡∏ô‡∏ê‡∏≤‡∏ô
    
    return processed_df

# ‡∏Ñ‡∏≥‡∏ô‡∏ß‡∏ì technical indicators
df = calculate_technical_indicators(df)

print(f"\nüìä Indicators summary:")
print(f"  Total columns: {len(df.columns)}")
print(f"  Indicators: {[col for col in df.columns if col not in ['timestamp', 'tic', 'open', 'high', 'low', 'close', 'volume']]}")

## Cell 4: Normalize ‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•

In [ ]:
def normalize_data(df):
    """
    Normalize ‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏£‡∏≤‡∏Ñ‡∏≤‡πÅ‡∏•‡∏∞ indicators
    """
    print("üìä Normalizing data...")
    
    # ‡πÅ‡∏¢‡∏Å‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏µ‡πà‡πÑ‡∏°‡πà‡∏ï‡πâ‡∏≠‡∏á normalize
    non_numeric_cols = ['timestamp', 'tic']
    numeric_cols = [col for col in df.columns if col not in non_numeric_cols]
    
    # ‡∏™‡∏£‡πâ‡∏≤‡∏á scaler
    scaler = MinMaxScaler()
    
    # Normalize ‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏™‡∏≥‡∏´‡∏£‡∏±‡∏ö‡πÅ‡∏ï‡πà‡∏•‡∏∞ symbol
    normalized_data = pd.DataFrame()
    
    for symbol in df['tic'].unique():
        print(f"\nNormalizing {symbol}...")
        symbol_data = df[df['tic'] == symbol].copy()
        
        # Normalize ‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ï‡∏±‡∏ß‡πÄ‡∏•‡∏Ç
        numeric_data = symbol_data[numeric_cols]
        normalized_numeric = scaler.fit_transform(numeric_data)
        
        # ‡∏™‡∏£‡πâ‡∏≤‡∏á DataFrame ‡πÉ‡∏´‡∏°‡πà
        normalized_symbol_data = pd.DataFrame(normalized_numeric, columns=numeric_cols)
        normalized_symbol_data[non_numeric_cols] = symbol_data[non_numeric_cols]
        
        # ‡πÄ‡∏û‡∏¥‡πà‡∏°‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏µ‡πà normalize ‡πÅ‡∏•‡πâ‡∏ß
        normalized_data = pd.concat([normalized_data, normalized_symbol_data])
        
        print(f"‚úÖ Normalized {len(symbol_data)} rows")
    
    # ‡πÄ‡∏£‡∏µ‡∏¢‡∏á‡∏•‡∏≥‡∏î‡∏±‡∏ö‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
    normalized_data = normalized_data.sort_values(['timestamp', 'tic']).reset_index(drop=True)
    
    print("\n‚úÖ Data normalization completed")
    return normalized_data

# Normalize ‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
df = normalize_data(df)

print(f"\nüìä Normalized data summary:")
print(f"  Total rows: {len(df)}")
print(f"  Total columns: {len(df.columns)}")
print(f"  Numeric columns: {[col for col in df.columns if col not in ['timestamp', 'tic']]}")

## Cell 5: ‡πÅ‡∏ö‡πà‡∏á‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡πÅ‡∏•‡∏∞‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å

In [ ]:
def split_data(df):
    """
    ‡πÅ‡∏ö‡πà‡∏á‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡πÄ‡∏õ‡πá‡∏ô Training/Validation/Testing
    """
    print("üìä Splitting data...")
    
    # ‡πÅ‡∏ö‡πà‡∏á‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ï‡∏≤‡∏°‡∏™‡∏±‡∏î‡∏™‡πà‡∏ß‡∏ô 70:15:15
    total_len = len(df)
    train_size = int(total_len * 0.7)
    val_size = int(total_len * 0.15)
    
    train_df = df.iloc[:train_size].reset_index(drop=True)
    val_df = df.iloc[train_size:train_size + val_size].reset_index(drop=True)
    test_df = df.iloc[train_size + val_size:].reset_index(drop=True)
    
    print(f"‚úÖ Data split completed:")
    print(f"  Training set: {len(train_df)} rows ({len(train_df)/total_len*100:.1f}%)")
    print(f"  Validation set: {len(val_df)} rows ({len(val_df)/total_len*100:.1f}%)")
    print(f"  Testing set: {len(test_df)} rows ({len(test_df)/total_len*100:.1f}%)")
    
    return train_df, val_df, test_df

def save_processed_data(df, train_df, val_df, test_df):
    """
    ‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏µ‡πà‡∏õ‡∏£‡∏∞‡∏°‡∏ß‡∏•‡∏ú‡∏•‡πÅ‡∏•‡πâ‡∏ß
    """
    print("üíæ Saving processed data...")
    
    # ‡∏™‡∏£‡πâ‡∏≤‡∏á timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # ‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡∏ó‡∏±‡πâ‡∏á‡∏´‡∏°‡∏î
    pickle_file = os.path.join(PROCESSED_DIR, f"processed_crypto_data_{timestamp}.pkl")
    with open(pickle_file, 'wb') as f:
        pickle.dump(df, f)
    print(f"‚úÖ Saved full data to {pickle_file}")
    
    # ‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•‡πÅ‡∏¢‡∏Å‡∏™‡πà‡∏ß‡∏ô
    train_file = os.path.join(PROCESSED_DIR, f"train_data_{timestamp}.pkl")
    val_file = os.path.join(PROCESSED_DIR, f"val_data_{timestamp}.pkl")
    test_file = os.path.join(PROCESSED_DIR, f"test_data_{timestamp}.pkl")
    
    with open(train_file, 'wb') as f:
        pickle.dump(train_df, f)
    with open(val_file, 'wb') as f:
        pickle.dump(val_df, f)
    with open(test_file, 'wb') as f:
        pickle.dump(test_df, f)
    
    print(f"‚úÖ Saved split data to:")
    print(f"  - {train_file}")
    print(f"  - {val_file}")
    print(f"  - {test_file}")
    
    # ‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å‡πÑ‡∏ü‡∏•‡πå‡∏•‡πà‡∏≤‡∏™‡∏∏‡∏î
    latest_files = {
        'full': os.path.join(PROCESSED_DIR, "processed_crypto_data.pkl"),
        'train': os.path.join(PROCESSED_DIR, "train_data.pkl"),
        'val': os.path.join(PROCESSED_DIR, "val_data.pkl"),
        'test': os.path.join(PROCESSED_DIR, "test_data.pkl")
    }
    
    for key, file_path in latest_files.items():
        if os.path.exists(file_path):
            os.remove(file_path)
        
        if key == 'full':
            df.to_csv(file_path.replace('.pkl', '.csv'), index=False)
        else:
            eval(f"{key}_df").to_csv(file_path.replace('.pkl', '.csv'), index=False)
    
    print(f"\n‚úÖ Saved latest files to:")
    for key, file_path in latest_files.items():
        print(f"  - {file_path.replace('.pkl', '.csv')}")

# ‡πÅ‡∏ö‡πà‡∏á‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
train_df, val_df, test_df = split_data(df)

# ‡∏ö‡∏±‡∏ô‡∏ó‡∏∂‡∏Å‡∏Ç‡πâ‡∏≠‡∏°‡∏π‡∏•
save_processed_data(df, train_df, val_df, test_df)

print(f"\n   Data preprocessing completed successfully!")
print(f"üìä Final data summary:")
print(f"  Total rows: {len(df)}")
print(f"  Total columns: {len(df.columns)}")
print(f"  Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"  Symbols: {df['tic'].unique()}")